# Data Scraping

This notebook contains a script that can be used to
scrape plot summaries en-masse from the IMDb website.

## Reading in the MovieLens Datasets

We want to scrape the plots for every movie in the dataset. 
The movielens 20m dataset includes an `imdbId` field, which we can use to look up the corresponding movie on the IMDb website. 

In [1]:
import numpy as np
import pandas as pd

In [2]:
movies_20m = pd.read_csv('data/movielens-20m/movies.csv', names=['movieid', 'title', 'genres'])
movies_1m = pd.read_csv('data/movielens-1m/movies.dat', delimiter='::', engine='python', 
                        header=None, names=['movieId', 'title', 'genres'])

In [3]:
titles_20m = set(movies_20m['title'])
titles_1m = set(movies_1m['title'])
print(len(titles_1m - titles_20m))
print(len(titles_20m - titles_1m))
del titles_1m
del titles_20m

543
23923


Looks like the 1m movie dataset isn't a strict subset of the 20m dataset.

Do the IDs match?

In [ ]:
common_movies = pd.merge(movies_1m, movies_20m, on='title', 
                         how='inner', suffixes=('_1m', '_20m'))
common_movies.head()

In [ ]:
unmatched_entries = common_movies.loc[common_movies['movieid_1m'] != common_movies['movieid_20m']]
print(unmatched_entries[['title', 'movieid_1m', 'movieid_20m']])

Almost a perfect match.
This tells us that we can use the 20m dataset's plot summaries to get the 1m dataset's plots as well. 

Now to fetch the plots themselves. 

# Movie Scraping

## 1m and 20m: Scraping the Intersection

These scrapers will take a while to run. Their reliability is also unknown. 
So for now, let's just scrape the movies at the intersection of the two datasets (1m $\cap$ 20m). 
This is easy to scrape (it inherits 20m's imdbID field), and keeps our output flexible (we can use it for either dataset later on).

The movie's links will be stored in `./plots.tsv`, as a tab-separated-value data file. 

In [ ]:
links_20m = pd.read_csv('data/movielens-20m/links.csv', header=0,
                        names=['movieid_20m', 'imdbid', 'tmdbid'])

common_movies = pd.merge(common_movies, links_20m, on='movieid_20m', how='inner')
common_movies.head()

In [ ]:
common_movies['plot'] = ""
common_movies.to_csv('common_movies.csv', index=False)

common_movies.head()

In [ ]:
import imdb
import csv

con = imdb.IMDb()

with open('plots.tsv', 'a') as f: 
    writer = csv.writer(f, delimiter='\t', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(len(common_movies)): 
        row = common_movies.loc[i]

        if row['plot'] != "": 
            # If we've already scraped this plot, move on. 
            print(f"Not fetching {row['imdbid']} {row['title']}, already in database.")
            writer.writerow([row['imdbid'], row['plot']])
        
        else: 
            print(imdbid, row['title'])
                         
            # Try to fetch a plot. If we can't find one, store an empty entry in our table.  
            movie = con.get_movie(row['imdbid'], info=['plot'])
            try: 
                plot = movie['plot'][0]
            except (KeyError, IndexError):
                plot = ""
        
            # Write to our output modes. 
            writer.writerow([row['imdbid'], plot])
            common_movies.at[i, 'plot'] = plot

con.close()

## 20m: Scraping it All

Now to scrape the remainder of the dataset. We'll pull in the data from IMDb, and add it to `plots.tsv`.

In [ ]:
# Make sure to read the IDs as strings. Otherwise, we'll drop the leading zeroes. 
links_20m = pd.read_csv('data/movielens-20m/links.csv', 
                        dtype={'movieId': 'string', 'imdbId': 'string', 'tmdbId': 'string'})
processed_movies = pd.read_csv('plots.tsv', delimiter='\t', quotechar='|', 
                        dtype={'imdbId': 'string'})
remaining_ids = set(links_20m['imdbId']) - set(processed_movies['imdbId'])
num_remaining_ids = len(remaining_ids)
print(f'{num_remaining_ids} movies remain to be processed.')

with open('plots.tsv', 'a') as f: 
    writer = csv.writer(f, delimiter='\t', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    for imdbid in remaining_ids:  
        num_remaining_ids -= 1
        if num_remaining_ids % 100 == 1: 
            print(f'{num_remaining_ids} movies remain.')

        movie = con.get_movie(imdbid, info=['plot'])
        try: 
            plot = movie['plot'][0]
        except (KeyError, IndexError):
            plot = ""
        
        # Write to our output modes. 
        writer.writerow([imdbid, plot])

Hoo boy. That took a while, didn't it?

In [ ]:
links_20m = pd.read_csv('data/movielens-20m/links.csv', 
                        dtype={'movieId': 'string', 'imdbId': 'string', 'tmdbId': 'string'})
plots_20m = pd.read_csv('data/movielens-20m/plots.tsv', 
                        delimiter='\t', quotechar='|', dtype={'imdbId': 'string', 'plot': 'string'})
movies_20m = pd.read_csv('data/movielens-20m/movies.csv', 
                        dtype={'movieId': 'string', 'title': 'string', 'genres': 'string'})

print(len(links_20m), len(plots_20m), len(movies_20m))

In [ ]:
# Join into the output CSV

union_20m = pd.merge(movies_20m, links_20m, on='movieId', how='inner')
assert len(union_20m) == len(links_20m)
assert len(union_20m) == len(movies_20m)

union_20m.head()

In [ ]:
union_20m = pd.merge(union_20m, plots_20m, on='imdbId', how='inner')
assert len(union_20m) == len(links_20m)

union_20m.head()

In [ ]:
union_20m.to_csv('movies.csv', index=False)

## 1m Dataset: Getting all Plot Summaries

In [10]:
movies_1m = pd.read_csv('data/movielens-1m/movies.dat', delimiter='::', engine='python', 
                        dtype={'movieId': 'string', 'title': 'string', 'genres': 'string'},
                        header=None, names=['movieId', 'title', 'genres'])
movieplots_20m = pd.read_csv('data/movielens-20m/movie-plots.csv', 
                        dtype={'movieId': 'string', 'title': 'string', 'genres': 'string', 
                              'imdbId': 'string', 'tmdbId': 'string'})

In [11]:
movies_1m.head()

,movieId,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
movieplots_20m.head()

,movieId,title,genres,imdbId,tmdbId,plot
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0114709,862,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji (1995),Adventure|Children|Fantasy,0113497,8844,When two kids find and play a magical board ga...
2,3,Grumpier Old Men (1995),Comedy|Romance,0113228,15602,John and Max resolve to save their beloved bai...
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0114885,31357,"Based on Terry McMillan's novel, this film fol..."
4,5,Father of the Bride Part II (1995),Comedy,0113041,11862,George Banks must deal not only with the pregn...


Hypothesis -- the movie IDs in both fields are the same. 
Michael has verified this. 

In [13]:
union = pd.merge(movies_1m, movieplots_20m.drop(columns=['title', 'genres', 'imdbId', 'tmdbId']),
                 suffixes=('_1m', '_20m'), on='movieId', how='left', validate='one_to_one')

union.head()

,movieId,title,genres,plot
0,1,Toy Story (1995),Animation|Children's|Comedy,A cowboy doll is profoundly threatened and jea...
1,2,Jumanji (1995),Adventure|Children's|Fantasy,When two kids find and play a magical board ga...
2,3,Grumpier Old Men (1995),Comedy|Romance,John and Max resolve to save their beloved bai...
3,4,Waiting to Exhale (1995),Comedy|Drama,"Based on Terry McMillan's novel, this film fol..."
4,5,Father of the Bride Part II (1995),Comedy,George Banks must deal not only with the pregn...


Notice how some movies are unique to the 1M dataset. We'll have to fetch these separately if we want a list of the 1M dataset's movies. What movies are we missing? 

In [14]:
missing_movies = union.loc[union['plot'].isnull()].set_index('movieId')
print(missing_movies)
print(missing_movies.at['720', 'title'])

                                                     title  \
movieId                                                      
557                                      Mamma Roma (1962)   
578                            Hour of the Pig, The (1993)   
644                                   Happy Weekend (1996)   
669                                       Aparajito (1956)   
720      Wallace & Gromit: The Best of Aardman Animatio...   
730                                   Low Life, The (1994)   
739                                       Honigmond (1996)   
752                           Vermont Is For Lovers (1992)   
770                                     Costa Brava (1946)   
791      Last Klezmer: Leopold Kozlowski, His Life and ...   
811                               Bewegte Mann, Der (1994)   
863                             Celestial Clockwork (1994)   
978              Blue Angel, The (Blaue Engel, Der) (1930)   
1108                                 Prerokbe Ognja (1995)   
1155    

We should now scrape these from the IMDb website by hand, and put them in `missingno.csv`.

In [15]:
manual = pd.read_csv('missingno.csv', dtype={'movieId': 'string', 'plot': 'string'})
manual.head()
print(len(manual))

42


In [24]:
# Now to fill in the missing plot summaries in the 1mil dataset. 

for i, row in manual.iterrows(): 
    union.loc[row['movieId'] == union['movieId'], 'plot'] = row['plot']

AttributeError: 'str' object has no attribute 'isnull'

In [25]:
missing_movies = union.loc[union['plot'].isnull()]
missing_movies

,movieId,title,genres,plot
639,644,Happy Weekend (1996),Comedy,<NA>
730,739,Honigmond (1996),Comedy,<NA>
742,752,Vermont Is For Lovers (1992),Comedy|Romance,<NA>
760,770,Costa Brava (1946),Drama,<NA>
781,791,"Last Klezmer: Leopold Kozlowski, His Life and ...",Documentary,<NA>
1299,1319,Kids of Survival (1993),Documentary,<NA>
1611,1657,Wonderland (1997),Documentary,<NA>
1658,1706,Harlem River Drive (1996),Drama,<NA>
1718,1774,Mass Transit (1998),Comedy|Drama,<NA>
2535,2604,Let it Come Down: The Life of Paul Bowles (1998),Documentary,<NA>


Why are we getting a null value? 

Ah, it's because empty plot strings are being treated as null. 
Let's replace null values with the empty string. 

In [26]:
union.fillna('', inplace=True)

In [27]:
union.loc[union['plot'].isnull()]

,movieId,title,genres,plot


We've now finished scraping the plots. Write the new 1m database (with plots) to `movies_1m.csv`. 

In [28]:
union.to_csv('movies_1m.csv', index=False)